<a href="https://colab.research.google.com/github/SahilCarterr/FinetuningLLM/blob/main/distill_bert_nueral_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 1. Data Preprocessing

In [ ]:
import pandas as pd
df = pd.read_csv('/content/data_cleaned2jul_witutL.csv')
unique_topics = df['topic'].unique()
topic_mapping = {topic: i for i, topic in enumerate(unique_topics)}

# Apply the mapping to the 'topic' column
df['topic'] = df['topic'].map(topic_mapping)

# Display the updated DataFrame and the mapping of unique strings to numbers
print("Mapping of unique strings to numbers:", topic_mapping)
import pandas as pd
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Save the train and test datasets to new CSV files
train_data.to_csv('train.csv', index=False)
test_data.to_csv('test.csv', index=False)

print("Data has been split and saved to train.csv and test.csv.")


Mapping of unique strings to numbers: {'Dating and Relationships': 0, 'Random': 1, 'Movies and TV': 2, 'Music': 3, 'Technology and Internet': 4, 'Philosophy and Religion': 5, 'technology and Internet': 6, 'Law and Order': 7, 'Health and Wellness': 8, 'Food and Drinks': 9, 'Fashion and Makeup': 10, 'health and Wellness': 11, 'Gaming': 12, 'Sports': 13, 'fashion and Makeup': 14, 'sports': 15, 'Anime': 16, 'anime': 17, 'Finance': 18, 'Travel and Geography': 19, 'Pets and Animals': 20, 'gaming': 21, 'random': 22, 'music': 23}
Data has been split and saved to train.csv and test.csv.


In [ ]:
!pip -q install pyarrow datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.


##### 1.1 Converting to Pytorch Dataset

In [ ]:
from datasets import load_dataset
data_files = {"train": "/content/train.csv", "test": "/content/test.csv"}
dataset = load_dataset("csv", data_files=data_files)
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'topic'],
        num_rows: 1260
    })
    test: Dataset({
        features: ['question', 'topic'],
        num_rows: 315
    })
})

##### 1.2 Making a tokenized Dataset

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding
checkpoint = "distilbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(examples):
    # Ensure inputs are strings, handle potential None or other non-string types
    questions = [str(q) if q is not None else "" for q in examples['question']]
    return tokenizer(questions,truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/1260 [00:00<?, ? examples/s]

Map:   0%|          | 0/315 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["question"])
tokenized_datasets = tokenized_datasets.rename_column("topic", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'attention_mask']

##### 1.3 Converting Data to batches

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=8, collate_fn=data_collator
)

#### 2. Creating the Model

In [ ]:
# Importing the libraries needed
import pandas as pd
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer

In [ ]:

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 1e-05
#tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')

In [ ]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model.

class DistillBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistillBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 24)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [ ]:
model = DistillBERTClass()
model.to(device)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

DistillBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(i

##### 2.1 Initializing the Loss and optimizer

In [ ]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
# Function to calcuate the accuracy of the model

def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

#### 3 Training the Model

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in enumerate(train_dataloader, 0):
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        targets = data['labels'].to(device, dtype = torch.long)

        outputs = model(ids, mask)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accu(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)

        if _%10==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return

In [ ]:
EPOCHS=2
for epoch in range(EPOCHS):
    train(epoch)

Training Loss per 5000 steps: 3.271740436553955
Training Accuracy per 5000 steps: 0.0
Training Loss per 5000 steps: 3.2032208876176314
Training Accuracy per 5000 steps: 1.1363636363636365
Training Loss per 5000 steps: 3.1238386631011963
Training Accuracy per 5000 steps: 13.69047619047619
Training Loss per 5000 steps: 3.0610107529547905
Training Accuracy per 5000 steps: 18.951612903225808
Training Loss per 5000 steps: 3.0148479822205334
Training Accuracy per 5000 steps: 21.646341463414632
Training Loss per 5000 steps: 2.935853457918354
Training Accuracy per 5000 steps: 25.245098039215687
Training Loss per 5000 steps: 2.8783816118709376
Training Accuracy per 5000 steps: 27.25409836065574
Training Loss per 5000 steps: 2.8408978337972934
Training Accuracy per 5000 steps: 28.169014084507044
Training Loss per 5000 steps: 2.801869833910907
Training Accuracy per 5000 steps: 29.166666666666668
Training Loss per 5000 steps: 2.7858931179885027
Training Accuracy per 5000 steps: 29.395604395604394


#### 4. Evaluating Model

In [ ]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0
    tr_loss = 0  # Initialize tr_loss here
    nb_tr_steps = 0
    nb_tr_examples = 0 # Initialize these variables as well
    with torch.no_grad():
        for _, data in enumerate(eval_dataloader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            targets = data['labels'].to(device, dtype = torch.long)
            outputs = model(ids, mask).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accu(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)

            if _%10==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 10 steps: {loss_step}")
                print(f"Validation Accuracy per 10 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")

    return epoch_accu

In [ ]:
acc = valid(model, eval_dataloader)
print("Accuracy on test data = %0.2f%%" % acc)

Validation Loss per 10 steps: 2.044330596923828
Validation Accuracy per 10 steps: 50.0
Validation Loss per 10 steps: 2.3705007162961094
Validation Accuracy per 10 steps: 40.90909090909091
Validation Loss per 10 steps: 2.3594733760470437
Validation Accuracy per 10 steps: 41.666666666666664
Validation Loss per 10 steps: 2.355185789446677
Validation Accuracy per 10 steps: 41.935483870967744
Validation Loss Epoch: 2.3509431004524233
Validation Accuracy Epoch: 40.317460317460316
Accuracy on test data = 40.32%
